In [10]:
import numpy as np
import MyML.helper.partition as myPart
import MyML.cluster.K_Means3 as myKM
import MyML.utils.sparse as mySparse
import MyML.cluster.eac as eac

In [6]:
from sklearn.datasets import make_blobs

In [29]:
data,gt = make_blobs(n_samples=5000, n_features=2, centers=6)
data = data.astype(np.float32)
n_samples = data.shape[0]

In [19]:
generator = myKM.K_Means(cuda_mem="manual")
generator._MAX_THREADS_BLOCK = 256

In [20]:
sqrt_rule = lambda n: map(int, map(np.ceil, [np.sqrt(n) / 2, np.sqrt(n)]))
n_clusters = sqrt_rule(data.shape[0])

In [22]:
%time ensemble = myPart.generateEnsemble(data, generator, n_clusters, npartitions=100)

CPU times: user 1.8 s, sys: 140 ms, total: 1.94 s
Wall time: 1.94 s


In [28]:
bgs = mySparse._compute_max_assocs_from_ensemble(ensemble)
max_assocs = bgs * 3

In [39]:
mat0 = mySparse.EAC_CSR(n_samples=n_samples, max_assocs=max_assocs)
%time mat0._update_ensemble(ensemble)
%time mat0._condense()

CPU times: user 14.7 s, sys: 14.6 ms, total: 14.7 s
Wall time: 14.7 s
CPU times: user 193 ms, sys: 3.92 ms, total: 196 ms
Wall time: 196 ms


In [40]:
mat1 = mySparse.EAC_CSR(n_samples=n_samples, max_assocs=max_assocs)
mat1.sort_mode = "online"
mat1.update_cluster_function = mySparse.update_cluster_sorted
%time mat1._update_ensemble(ensemble)
%time mat1._condense()

CPU times: user 6.85 s, sys: 76.1 ms, total: 6.92 s
Wall time: 6.9 s
CPU times: user 12.5 ms, sys: 11 µs, total: 12.5 ms
Wall time: 12.6 ms


In [41]:
mat2 = mySparse.EAC_CSR(n_samples=n_samples, max_assocs=max_assocs)
mat2.sort_mode = "online"
mat2.update_cluster_function = mySparse.update_cluster_sorted_simple
%time mat2._update_ensemble(ensemble)
%time mat2._condense()

CPU times: user 6.39 s, sys: 19.8 ms, total: 6.41 s
Wall time: 6.39 s
CPU times: user 12.3 ms, sys: 4 µs, total: 12.3 ms
Wall time: 12.3 ms


In [42]:
mat3 = mySparse.EAC_CSR(n_samples=n_samples, max_assocs=max_assocs)
mat3.sort_mode = "online"
mat3.update_cluster_function = mySparse.update_cluster_sorted_surgical
%time mat3._update_ensemble(ensemble)
%time mat3._condense()

CPU times: user 6.31 s, sys: 35.6 ms, total: 6.35 s
Wall time: 6.33 s
CPU times: user 46.6 ms, sys: 0 ns, total: 46.6 ms
Wall time: 46.6 ms


In [60]:
eacEst = eac.EAC(n_samples=n_samples)
%time eacEst.fit(ensemble)
eacEst._getAssocsDegree()

CPU times: user 859 ms, sys: 12.1 ms, total: 871 ms
Wall time: 867 ms


In [52]:
print "check matrices"
print "mat0: ", (eacEst._coassoc == mat0.todense()).all()
print "mat1: ", (eacEst._coassoc == mat1.todense()).all()
print "mat2: ", (eacEst._coassoc == mat2.todense()).all()
print "mat3: ", (eacEst._coassoc == mat3.todense()).all()

check matrices
mat0:  True
mat1:  True
mat2:  False
mat3:  False


In [63]:
print "eac:  ", eacEst.nnz
print "mat0: ",mat0.nnz
print "mat1: ",mat1.nnz
print "mat2: ",mat2.nnz
print "mat3: ",mat3.nnz

eac:   3107432
mat0:  3107432
mat1:  3107432
mat2:  3107432
mat3:  12495000


In [67]:
(eacEst._coassoc == mat2.todense()).sum()

22080225